# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [15]:
TRIAL_LABELS_DF = pd.read_excel("./proc/rce2_finalized_trial_labels.xlsx")

In [16]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_03_spectral_bands.pkl")

In [17]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [18]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [19]:
BIN_TO_VELOCITY = {0: "0 to 2.5cm/s", 1: "2.5 to 5cm/s", 2: "5 to 10 cm/s", 3: "10cm/s+"}

In [20]:
ALL_BANDS = ["theta", "beta", "gamma"]
BAND_TO_FREQ = {"theta": {"low_freq": 6, "high_freq": 11}, "beta": {"low_freq": 20, "high_freq": 31}, "gamma": {"low_freq": 30, "high_freq": 51}}
BAND_TO_FREQ = {"theta": (6,11), "beta": (20,31), "gamma": (30,51)}

In [21]:
GROUPINGS = "trial_outcome"
# GROUPINGS = "competition_closeness"

In [22]:
FONTSIZE=20

In [23]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [24]:
def generate_pairs(lst):
    pairs = []
    n = len(lst)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((lst[i], lst[j]))
    return pairs

In [25]:
def nested_dict():
    return defaultdict(dict)

In [26]:
def mean_of_counter(counter):
    # Extract values from the Counter and calculate the mean
    values = list(counter.values())
    return sum(values) / len(values) if values else 0  # Prevent division by zero

In [27]:
def adjust_lightness(color, amount=0.5):
    """
    Adjusts the lightness of the given color by the provided amount.
    :param color: Input color in some format that matplotlib's `to_rgb` can handle.
    :param amount: Amount to adjust. > 1 for lighter, < 1 for darker.
    :return: Adjusted color.
    """
    import colorsys
    c = mcolors.to_rgb(color)
    h, l, s = colorsys.rgb_to_hls(*c)
    return colorsys.hls_to_rgb(h, max(0, min(1, amount * l)), s)

In [28]:
def generate_gradient_colors(base_color, num_colors=10):
    """
    Generate gradient colors based on a base color.
    
    :param base_color: The base color to create a gradient from.
    :param num_colors: The number of gradient colors to generate.
    :return: List of colors.
    """
    # Convert base color to HLS (Hue, Lightness, Saturation)
    h, l, s = colorsys.rgb_to_hls(*mcolors.to_rgb(base_color))

    # Determine the step size for lightness
    middle_index = num_colors // 2
    lightness_step = l / (middle_index if middle_index > 0 else 1)

    colors = []
    for i in range(num_colors):
        if i < middle_index:
            # For the lighter colors (before the middle)
            new_lightness = l + (middle_index - i) * lightness_step
        elif i == middle_index:
            # The middle color remains the base color
            new_lightness = l
        else:
            # For the darker colors (after the middle)
            new_lightness = l - (i - middle_index) * lightness_step

        # Ensure new lightness is within the valid range [0, 1]
        new_lightness = max(0, min(new_lightness, 1))

        new_color = colorsys.hls_to_rgb(h, new_lightness, s)
        colors.append(new_color)

    return colors

In [29]:
def get_significance_stars_from_p_value(p_value, number_of_comparisons=3):
    """
    """
    if p_value <= 0.001 / number_of_comparisons:
        return "***"
    elif p_value <= 0.01 / number_of_comparisons:
        return "***"
    elif p_value <= 0.05 / number_of_comparisons:
        return "*"
    else:
        return ""

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [30]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [31]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject',
       ...
       'mPFC_RMS_filtered_power_theta', 'mPFC_RMS_filtered_power_gamma',
       'MD_RMS_filtered_power_theta', 'MD_RMS_filtered_power_gamma',
       'LH_RMS_filtered_power_theta', 'LH_RMS_filtered_power_gamma',
       'BLA_RMS_filtered_power_theta', 'BLA_RMS_filtered_power_gamma',
       'vHPC_RMS_filtered_power_theta', 'vHPC_RMS_filtered_power_gamma'],
      dtype='object', length=110)

In [32]:
TRIAL_LABELS_DF.columns

Index(['tracked_subject', 'box_number', 'sleap_name', 'video_name',
       'current_subject', 'tone_start_frame', 'reward_start_frame',
       'tone_stop_frame', 'condition', 'competition_closeness', 'notes',
       'experiment', 'session_dir', 'all_subjects', 'tone_start_timestamp',
       'tone_stop_timestamp', 'trial_label'],
      dtype='object')

## Merging the labels and LFP spectral

In [33]:
TRIAL_LABELS_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp,trial_label
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,win
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1125,1225,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,lose
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,lose
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,3519,3619,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,win
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",5826740,6026737,lose


In [34]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC_RMS_filtered_power_theta,mPFC_RMS_filtered_power_gamma,MD_RMS_filtered_power_theta,MD_RMS_filtered_power_gamma,LH_RMS_filtered_power_theta,LH_RMS_filtered_power_gamma,BLA_RMS_filtered_power_theta,BLA_RMS_filtered_power_gamma,vHPC_RMS_filtered_power_theta,vHPC_RMS_filtered_power_gamma
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.00652, 0.002836, 0.002962, 0.00776, 0.01208...","[0.000125, 0.00016, 0.0001565, 0.0001225, 0.00...","[0.01796, 0.01675, 0.01249, 0.01534, 0.013664,...","[0.0003269, 0.0002108, 0.0003424, 0.0004702, 0...","[0.0123, 0.012405, 0.01134, 0.010544, 0.009544...","[0.0004094, 0.0003133, 0.000283, 0.0003424, 0....","[0.01605, 0.01243, 0.01147, 0.01507, 0.014824,...","[0.000981, 0.001048, 0.001005, 0.0007415, 0.00...","[0.006912, 0.0188, 0.03604, 0.01837, 0.010605,...","[0.000249, 0.000289, 0.0002723, 0.0003161, 0.0..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.00652, 0.002836, 0.002962, 0.00776, 0.01208...","[0.000125, 0.00016, 0.0001565, 0.0001225, 0.00...","[0.01796, 0.01675, 0.01249, 0.01534, 0.013664,...","[0.0003269, 0.0002108, 0.0003424, 0.0004702, 0...","[0.0123, 0.012405, 0.01134, 0.010544, 0.009544...","[0.0004094, 0.0003133, 0.000283, 0.0003424, 0....","[0.01605, 0.01243, 0.01147, 0.01507, 0.014824,...","[0.000981, 0.001048, 0.001005, 0.0007415, 0.00...","[0.006912, 0.0188, 0.03604, 0.01837, 0.010605,...","[0.000249, 0.000289, 0.0002723, 0.0003161, 0.0..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.01492, 0.01654, 0.03436, nan, nan, 0.01553,...","[0.001747, 0.002792, 0.00246, nan, nan, 0.0031...","[0.01246, 0.01306, 0.0218, 0.0201, 0.01401, 0....","[0.000678, 0.0005302, 0.0004344, 0.000489, 0.0...","[0.01442, 0.01605, 0.02498, 0.02408, 0.0166, 0...","[0.0008626, 0.0006566, 0.000616, 0.000811, 0.0...","[0.013916, nan, nan, 0.02411, nan, nan, nan, n...","[0.003502, nan, nan, 0.003157, nan, nan, nan, ...","[0.026, nan, nan, nan, nan, nan, nan, nan, 0.0...","[0.003092, nan, nan, nan, nan, nan, nan, nan, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.01492, 0.01654, 0.03436, nan, nan, 0.01553,...","[0.001747, 0.002792, 0.00246, nan, nan, 0.0031...","[0.01246, 0.01306, 0.0218, 0.0201, 0.01401, 0....","[0.000678, 0.0005302, 0.0004344, 0.000489, 0.0...","[0.01442, 0.01605, 0.02498, 0.02408, 0.0166, 0...","[0.0008626, 0.0006566, 0.000616, 0.000811, 0.0...","[0.013916, nan, nan, 0.02411, nan, nan, nan, n...","[0.003502, nan, nan, 0.003157, nan, nan, nan, ...","[0.026, nan, nan, nan, nan, nan, nan, nan, 0.0...","[0.003092, nan, nan, nan, nan, nan, nan, nan, ..."


In [35]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [36]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)
TRIAL_LABELS_DF["current_subject"] = TRIAL_LABELS_DF["current_subject"].astype(str)

In [37]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1',
       '20230620_114347_standard_comp_

In [38]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.1',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.2',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.2',
       '20230617_115521_standard_comp_

In [39]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["current_subject", "video_name"], how="inner", suffixes=('', '_y'))

In [40]:
# TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col], errors="ignore")

In [41]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "_y" in col], errors="ignore")

In [42]:
TRIALS_AND_SPECTRAL_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,mPFC_RMS_filtered_power_theta,mPFC_RMS_filtered_power_gamma,MD_RMS_filtered_power_theta,MD_RMS_filtered_power_gamma,LH_RMS_filtered_power_theta,LH_RMS_filtered_power_gamma,BLA_RMS_filtered_power_theta,BLA_RMS_filtered_power_gamma,vHPC_RMS_filtered_power_theta,vHPC_RMS_filtered_power_gamma
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[nan, 0.01707, 0.03317, nan, nan, nan, nan, na...","[nan, 0.001803, 0.002707, nan, nan, nan, nan, ...","[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ..."
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[nan, 0.01707, 0.03317, nan, nan, nan, nan, na...","[nan, 0.001803, 0.002707, nan, nan, nan, nan, ...","[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ..."
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[nan, 0.01707, 0.03317, nan, nan, nan, nan, na...","[nan, 0.001803, 0.002707, nan, nan, nan, nan, ...","[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ..."
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[nan, 0.01707, 0.03317, nan, nan, nan, nan, na...","[nan, 0.001803, 0.002707, nan, nan, nan, nan, ...","[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ..."
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[nan, 0.01707, 0.03317, nan, nan, nan, nan, na...","[nan, 0.001803, 0.002707, nan, nan, nan, nan, ...","[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ..."


In [43]:
TRIALS_AND_SPECTRAL_DF.columns

Index(['tracked_subject', 'box_number', 'sleap_name', 'video_name',
       'current_subject', 'tone_start_frame', 'reward_start_frame',
       'tone_stop_frame', 'condition', 'competition_closeness',
       ...
       'mPFC_RMS_filtered_power_theta', 'mPFC_RMS_filtered_power_gamma',
       'MD_RMS_filtered_power_theta', 'MD_RMS_filtered_power_gamma',
       'LH_RMS_filtered_power_theta', 'LH_RMS_filtered_power_gamma',
       'BLA_RMS_filtered_power_theta', 'BLA_RMS_filtered_power_gamma',
       'vHPC_RMS_filtered_power_theta', 'vHPC_RMS_filtered_power_gamma'],
      dtype='object', length=123)

# Calculating the relevant timestamps

In [44]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 30 * 20000

In [45]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 20 * 20000

In [46]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

0     526742
1    2926740
2    5226740
3    7026736
4    8126734
Name: baseline_start_timestamp, dtype: int64

In [47]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

0     726742
1    3126740
2    5426740
3    7226736
4    8326734
Name: baseline_stop_timestamp, dtype: int64

In [48]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1126742
1    3526740
2    5826740
3    7626736
4    8726734
Name: tone_start_timestamp, dtype: int64

In [49]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    1326741
1    3726740
2    6026737
3    7826735
4    8926734
Name: tone_stop_timestamp, dtype: int64

- Filtering power

In [50]:
TRIALS_AND_SPECTRAL_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,MD_RMS_filtered_power_theta,MD_RMS_filtered_power_gamma,LH_RMS_filtered_power_theta,LH_RMS_filtered_power_gamma,BLA_RMS_filtered_power_theta,BLA_RMS_filtered_power_gamma,vHPC_RMS_filtered_power_theta,vHPC_RMS_filtered_power_gamma,baseline_start_timestamp,baseline_stop_timestamp
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ...",526742,726742
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ...",2926740,3126740
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ...",5226740,5426740
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ...",7026736,7226736
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[0.04553, 0.02853, nan, nan, nan, 0.03503, 0.0...","[0.001228, 0.001134, nan, nan, nan, 0.001013, ...","[nan, 0.02965, nan, nan, nan, 0.03806, nan, na...","[nan, 0.001294, nan, nan, nan, 0.000928, nan, ...","[nan, 0.0215, nan, nan, nan, nan, nan, nan, na...","[nan, 0.002539, nan, nan, nan, nan, nan, nan, ...","[nan, 0.01796, nan, nan, nan, nan, nan, nan, n...","[nan, 0.003302, nan, nan, nan, nan, nan, nan, ...",8126734,8326734


In [51]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [52]:
power_columns

['mPFC_trace_power_all_frequencies_all_windows',
 'MD_trace_power_all_frequencies_all_windows',
 'LH_trace_power_all_frequencies_all_windows',
 'BLA_trace_power_all_frequencies_all_windows',
 'vHPC_trace_power_all_frequencies_all_windows',
 'mPFC_RMS_power_all_frequencies_all_windows',
 'MD_RMS_power_all_frequencies_all_windows',
 'LH_RMS_power_all_frequencies_all_windows',
 'BLA_RMS_power_all_frequencies_all_windows',
 'vHPC_RMS_power_all_frequencies_all_windows',
 'mPFC_trace_filtered_power_all_frequencies_all_windows',
 'MD_trace_filtered_power_all_frequencies_all_windows',
 'LH_trace_filtered_power_all_frequencies_all_windows',
 'BLA_trace_filtered_power_all_frequencies_all_windows',
 'vHPC_trace_filtered_power_all_frequencies_all_windows',
 'mPFC_RMS_filtered_power_all_frequencies_all_windows',
 'MD_RMS_filtered_power_all_frequencies_all_windows',
 'LH_RMS_filtered_power_all_frequencies_all_windows',
 'BLA_RMS_filtered_power_all_frequencies_all_windows',
 'vHPC_RMS_filtered_power_

In [53]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_trace_power_all_frequencies
MD_trace_power_all_frequencies
LH_trace_power_all_frequencies
BLA_trace_power_all_frequencies
vHPC_trace_power_all_frequencies
mPFC_RMS_power_all_frequencies
MD_RMS_power_all_frequencies
LH_RMS_power_all_frequencies
BLA_RMS_power_all_frequencies
vHPC_RMS_power_all_frequencies
mPFC_trace_filtered_power_all_frequencies
MD_trace_filtered_power_all_frequencies
LH_trace_filtered_power_all_frequencies
BLA_trace_filtered_power_all_frequencies
vHPC_trace_filtered_power_all_frequencies
mPFC_RMS_filtered_power_all_frequencies
MD_RMS_filtered_power_all_frequencies
LH_RMS_filtered_power_all_frequencies
BLA_RMS_filtered_power_all_frequencies
vHPC_RMS_filtered_power_all_frequencies
mPFC_trace_power_theta
mPFC_trace_power_gamma
MD_trace_power_theta
MD_trace_power_gamma
LH_trace_power_theta
LH_trace_power_gamma
BLA_trace_power_theta
BLA_trace_power_gamma
vHPC_trace_power_theta
vHPC_trace_power_gamma
mPFC_RMS_power_theta
mPFC_RMS_power_gamma
MD_RMS_power_theta
MD_RMS_po

In [54]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_trace_power_all_frequencies
MD_trace_power_all_frequencies
LH_trace_power_all_frequencies
BLA_trace_power_all_frequencies
vHPC_trace_power_all_frequencies
mPFC_RMS_power_all_frequencies
MD_RMS_power_all_frequencies
LH_RMS_power_all_frequencies
BLA_RMS_power_all_frequencies
vHPC_RMS_power_all_frequencies
mPFC_trace_filtered_power_all_frequencies
MD_trace_filtered_power_all_frequencies
LH_trace_filtered_power_all_frequencies
BLA_trace_filtered_power_all_frequencies
vHPC_trace_filtered_power_all_frequencies
mPFC_RMS_filtered_power_all_frequencies
MD_RMS_filtered_power_all_frequencies
LH_RMS_filtered_power_all_frequencies
BLA_RMS_filtered_power_all_frequencies
vHPC_RMS_filtered_power_all_frequencies
mPFC_trace_power_theta
mPFC_trace_power_gamma
MD_trace_power_theta
MD_trace_power_gamma
LH_trace_power_theta
LH_trace_power_gamma
BLA_trace_power_theta
BLA_trace_power_gamma
vHPC_trace_power_theta
vHPC_trace_power_gamma
mPFC_RMS_power_theta
mPFC_RMS_power_gamma
MD_RMS_power_theta
MD_RMS_po

In [55]:
TRIALS_AND_SPECTRAL_DF["baseline_power_timestamps"]

0      [530000, 540000, 550000, 560000, 570000, 58000...
1      [2930000, 2940000, 2950000, 2960000, 2970000, ...
2      [5230000, 5240000, 5250000, 5260000, 5270000, ...
3      [7030000, 7040000, 7050000, 7060000, 7070000, ...
4      [8130000, 8140000, 8150000, 8160000, 8170000, ...
                             ...                        
962    [60080000, 60090000, 60100000, 60110000, 60120...
963    [61580000, 61590000, 61600000, 61610000, 61620...
964    [63580000, 63590000, 63600000, 63610000, 63620...
965    [65480000, 65490000, 65500000, 65510000, 65520...
966    [66480000, 66490000, 66500000, 66510000, 66520...
Name: baseline_power_timestamps, Length: 967, dtype: object

In [56]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [57]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [58]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace
mPFC_lfp_zscore
MD_lfp_zscore
LH_lfp_zscore
BLA_lfp_zscore
vHPC_lfp_zscore
mPFC_lfp_RMS
MD_lfp_RMS
LH_lfp_RMS
BLA_lfp_RMS
vHPC_lfp_RMS
mPFC_lfp_mask
MD_lfp_mask
LH_lfp_mask
BLA_lfp_mask
vHPC_lfp_mask
mPFC_lfp_trace_filtered
MD_lfp_trace_filtered
LH_lfp_trace_filtered
BLA_lfp_trace_filtered
vHPC_lfp_trace_filtered
mPFC_lfp_RMS_filtered
MD_lfp_RMS_filtered
LH_lfp_RMS_filtered
BLA_lfp_RMS_filtered
vHPC_lfp_RMS_filtered
power_calculation_frequencies
bas

In [59]:
raise ValueError()

ValueError: 

- Filtering coherence

In [ ]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
coherence_columns

In [ ]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [ ]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
granger_columns

In [ ]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

# Filtering out the SLEAP posed for during trials

In [ ]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col]

In [ ]:
sleap_columns

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_timestamps"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["subject_thorax_to_reward_port"].iloc[0].shape

In [ ]:
for col in sorted(sleap_columns):
    updated_item_col = "baseline_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "baseline_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp]

In [ ]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_subject_thorax_velocity"].apply(lambda x: np.isnan(x).any())]["baseline_subject_locations"]

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))